## SCRATCH

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load dataset
iris = load_iris()
X = iris.data  # 4 fitur
y = iris.target  # Label numerik (0, 1, 2)

# One-hot encoding label (karena kita ingin output dengan softmax)
def one_hot_encode(y, num_classes=3):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y] = 1
    return one_hot

y = one_hot_encode(y)

# Normalisasi data agar mean = 0, std = 1
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Membagi dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)  # Supaya hasilnya tetap sama
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Bobot layer 1
    b1 = np.zeros((1, hidden_size))  # Bias layer 1
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Bobot layer 2
    b2 = np.zeros((1, output_size))  # Bias layer 2
    return W1, b1, W2, b2

# Ukuran layer
input_size = 4      # Fitur dataset
hidden_size = 5     # Jumlah neuron di hidden layer
output_size = 3     # Jumlah kelas (Setosa, Versicolor, Virginica)

# Inisialisasi bobot dan bias
W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)


In [ ]:
# Fungsi aktivasi ReLU
def relu(Z):
    return np.maximum(0, Z)

# Fungsi aktivasi Softmax untuk output layer
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # Stabilitas numerik
    return expZ / np.sum(expZ, axis=1, keepdims=True)

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1  # Linear transform ke hidden layer
    A1 = relu(Z1)  # Aktivasi ReLU
    Z2 = np.dot(A1, W2) + b2  # Linear transform ke output layer
    A2 = softmax(Z2)  # Aktivasi Softmax
    
    # Menyimpan hasil untuk backward propagation
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache


In [ ]:
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]  # Jumlah sampel
    loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m  # Tambah epsilon agar tidak log(0)
    return loss


In [ ]:
# Fungsi turunan ReLU
def relu_derivative(Z):
    return Z > 0  # 1 jika Z > 0, 0 jika tidak

# Backpropagation
def backward_propagation(X, y_true, cache, W1, W2, learning_rate=0.01):
    m = X.shape[0]  # Jumlah sampel

    # Ambil hasil dari forward pass
    A1, A2 = cache["A1"], cache["A2"]

    # Hitung error di output layer
    dZ2 = A2 - y_true
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)

    # Hitung error di hidden layer
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(cache["Z1"])
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)

    # Update bobot dan bias
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2


In [ ]:
# Hyperparameter
epochs = 500
learning_rate = 0.01

# Training loop
for epoch in range(epochs):
    # Forward pass
    y_pred, cache = forward_propagation(X_train, W1, b1, W2, b2)
    
    # Compute loss
    loss = compute_loss(y_train, y_pred)

    # Backpropagation
    W1, b1, W2, b2 = backward_propagation(X_train, y_train, cache, W1, W2, learning_rate)

    # Print loss setiap 50 epoch
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


## TF

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)  # Reshape for OneHotEncoder

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalize input features

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)  # Convert labels to one-hot encoding

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build simple model
model = keras.Sequential([
    keras.layers.Dense(4, activation='relu', input_shape=(4,)),  # 1 hidden layer with 4 neurons
    keras.layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
